In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 3.1/390.3 MB 26.5 MB/s eta 0:00:15
   ---------------------------------------- 3.9/390.3 MB 9.4 MB/s eta 0:00:42
   ---------------------------------------- 4.5/390.3 MB 9.9 MB/s eta 0:00:39
    --------------------------------------- 6.0/390.3 MB 7.2 MB/s eta 0:00:54
    --------------------------------------- 7.1/390.3 MB 6.9 MB/s eta 0:00:56
    --------------------------------------- 7.9/390.3 MB 6.5 MB/s eta 0:00:59
    --------------------------------------- 8.7/390.3 MB 6.1 MB/s eta 0:01:03
    --------------------------------------- 9.7/390.3 MB 5.9 MB/s eta 0:01:05
   - -------------------------------------- 10.5/390.3 MB 5.7 MB/s eta 0:01:07
   - -------------------------------------- 11.3/390.3 MB 5.5 MB/s eta 0:01:09
   - -------------------------------------- 12.6/390.3 MB 5.6 MB/s e

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
pip install keras

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [6]:
pip install emoji

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   --------------------------------------- 590.6/590.6 kB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
data = pd.read_csv("C:\\Users\\navya\\Downloads\\emoji_data.csv", header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [3]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [4]:
X = data[0].values
Y = data[1].values

In [5]:
file = open("C:\\Users\\navya\\Downloads\\glove.6B.100d.txt", 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [6]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [12]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

10


In [11]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [10]:
# Clean `Y` to ensure all labels are integers
data = data[data[1].apply(lambda x: str(x).isdigit())]  # Filter out invalid labels
X = data[0].values
Y = data[1].values.astype(int)  # Convert to integers

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
Xtokens = tokenizer.texts_to_sequences(X)

# Determine maxlen if not manually set
maxlen = max(len(seq) for seq in Xtokens)
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')

# One-hot encode labels
Ytrain = to_categorical(Y)


In [13]:
print(Y)

[4 3 2 1 4 3 4 1 3 2 2 4 3 2 1 2 2 2 0 1 0 2 2 0 0 4 0 2 1 3 1 0 4 3 4 2 4
 2 3 0 2 2 2 3 2 2 3 2 3 0 2 0 2 3 2 4 1 3 3 0 0 3 2 0 3 0 2 2 4 2 2 0 0 2
 3 0 4 2 1 2 3 3 2 3 0 3 0 2 0 2 3 4 3 1 3 4 3 2 3 3 3 1 4 4 2 2 1 1 2 3 2
 3 4 2 3 0 2 0 0 4 3 4 2 3 2 3 4 2 1 2 4 3 1 3 2 3 2 2 3 3 2 4 0 0 0 3 0 0
 1 1 2 2 2 0 3 2 3 3 1 2 2 4 2 3 1 2]


In [14]:

embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [15]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [16]:
model = Sequential([
    Embedding(input_dim=len(word2index) + 1,
              output_dim=embed_size,
              weights=[embedding_matrix],
              trainable=False
             ),
    LSTM(units=16, return_sequences=True),
    LSTM(units=4),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.1405 - loss: 1.6069
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2247 - loss: 1.5991
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2833 - loss: 1.5896 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3329 - loss: 1.5841
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3510 - loss: 1.5672
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4086 - loss: 1.5590
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3811 - loss: 1.5525
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3655 - loss: 1.5482
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3960 - loss: 1.5256
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3500 - loss: 1.5115
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3216 - loss: 1.4915
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4093 - 

In [1]:
test = ["i hungry", "I love you", "dinner","1234","i love you mum","never talk to me again","Let us go play baseball","I cooked meat","ha ha ha lol"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

NameError: name 'tokenizer' is not defined